# Create the stations, lines, and travel times table for BART system

In [43]:
import csv

import math
import numpy as np
import pandas as pd

import psycopg2

import json

import neo4j

In [2]:
def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)
cursor = connection.cursor()

In [4]:
def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

In [24]:
# create the stations table and load data

connection.rollback()

query = """

drop table if exists stations; 
create table stations (
  station varchar(32),
  latitude numeric(9,6),
  longitude numeric(9,6),
  transfer_time numeric(3),
  primary key (station)
);


copy stations
from '/user/projects/project-3-peeti-ucberkeley/code/data/stations.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

my_read_csv_file("data/stations.csv", limit=10)

['station', 'latitude', 'longitude', 'transfer_time']
['12th Street', '37.803608', '-122.272006', '282']
['16th Street Mission', '37.764847', '-122.420042', '287']
['19th Street', '37.807869', '-122.26898', '67']
['24th Street Mission', '37.752', '-122.4187', '277']
['Antioch', '37.996281', '-121.783404', '0']
['Ashby', '37.853068', '-122.269957', '299']
['Balboa Park', '37.721667', '-122.4475', '48']
['Bay Fair', '37.697', '-122.1265', '63']
['Berryessa', '37.368361', '-121.874655', '288']

Printed  10 lines of  51 total lines.


In [23]:
#create lines table and load data

connection.rollback()

query = """

drop table if exists lines;

create table lines (
  line varchar(6),
  sequence numeric(2),
  station varchar(32),
  primary key (line, sequence)
);

copy lines
from '/user/projects/project-3-peeti-ucberkeley/code/data/lines.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

my_read_csv_file("data/lines.csv", limit=10)

['line', 'sequence', 'station']
['blue', '1', 'Dublin']
['blue', '2', 'West Dublin']
['blue', '3', 'Castro Valley']
['blue', '4', 'Bay Fair']
['blue', '5', 'San Leandro']
['blue', '6', 'Coliseum']
['blue', '7', 'Fruitvale']
['blue', '8', 'Lake Merritt']
['blue', '9', 'West Oakland']

Printed  10 lines of  115 total lines.


In [22]:
# create travel times table and load data

connection.rollback()

query = """

drop table if exists travel_times;
create table travel_times (
  station_1 varchar(32),
  station_2 varchar(32),
  travel_time numeric(3),
  primary key (station_1, station_2)
);

copy travel_times
from '/user/projects/project-3-peeti-ucberkeley/code/data/travel_times.csv' delimiter ',' NULL '' csv header;


"""


cursor.execute(query)

connection.commit()

my_read_csv_file("data/travel_times.csv", limit=10)

['station_1', 'station_2', 'travel_time']
['12th Street', '19th Street', '120']
['12th Street', 'Lake Merritt', '180']
['12th Street', 'West Oakland', '300']
['16th Street Mission', '24th Street Mission', '120']
['16th Street Mission', 'Civic Center', '180']
['19th Street', 'MacArthur', '180']
['24th Street Mission', 'Glen Park', '180']
['Antioch', 'Pittsburg Center', '420']
['Ashby', 'Downtown Berkeley', '180']

Printed  10 lines of  52 total lines.


In [19]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select station from stations
order by station

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station
0,12th Street
1,16th Street Mission
2,19th Street
3,24th Street Mission
4,Antioch
5,Ashby
6,Balboa Park
7,Bay Fair
8,Berryessa
9,Castro Valley


# Create peak product table

In [27]:
connection.rollback()

query = """

drop table if exists peak_product_mapping;
CREATE TABLE peak_product_mapping
( product_id numeric(3) primary key,
  peak_product_id numeric(12)) ;
  
copy peak_product_mapping
from '/user/projects/project-3-peeti-ucberkeley/code/data/peak_product_mapping.csv' delimiter ',' NULL '' csv header;
  

"""

cursor.execute(query)

connection.commit()

my_read_csv_file("data/peak_product_mapping.csv", 10)

['product_id', 'peak_product_id']
['1', '42314677']
['2', '42314678']
['3', '42314679']
['4', '42314780']
['5', '42314781']
['6', '42314782']
['7', '42314783']
['8', '42314784']

Printed  9 lines of  9 total lines.


# parse peak sales from json file to csv and create a table and load data 

In [35]:
def my_recursive_print_json(j, level = -1):
    "given a json object print it"
    
    level += 1
    
    spaces = "    "
    
    if type(j) is dict:
        dict_2_list = list(j.keys())
        for k in dict_2_list:
            print(spaces * level + k)
            my_recursive_print_json(j[k], level)
            
    elif type(j) is list:
        for (i, l) in enumerate(j):
            print(spaces * level + "[" + str(i) + "]")
            my_recursive_print_json(l, level)
                  
    else:
        print(spaces * level + "value:", str(j))
      

def my_read_nested_json(file_name):
    "given a file of json, read it and parse it meaningfully"
    
    f = open(file_name, "r")
    
    j = json.load(f)
    
    f.close
    
    
    my_recursive_print_json(j)
    
my_read_nested_json('data/peak_sales_2020_10_03.json')

# parse the data in csv

def my_extract_peak_sales_2020_nested_json_to_csv(file_name):
    "extract sales nested json to csv files"
    
    f = open(file_name, "r")
    j = json.load(f)
    f.close()   
    
    sale_json_list = []
    store_json_list  = []
    customer_json_list = []
    line_item_json_list = []
    
    for sale in j['sales']:
        
        sale_json = {}
        sale_json['sale_id'] = sale['sale_id']
        sale_json['sale_date'] = sale['sale_date']
        sale_json['sub_total'] = sale['sub_total']
        sale_json['tax'] = sale['tax']
        sale_json['total_amount'] = sale['total_amount']
        
        sale_json_list.append(sale_json)
        
        store_json = {}
        store_json['sale_id'] = sale['sale_id']
        store_json['location_id'] = sale['pickup_from']['location_id'] 
        store_json['name'] = sale['pickup_from']['name']
        store_json['street'] = sale['pickup_from']['street']
        store_json['city'] = sale['pickup_from']['city']
        store_json['state'] = sale['pickup_from']['state']
        store_json['zip'] = sale['pickup_from']['zip']
          
        store_json_list.append(store_json)
        
        customer_json = {}
        customer_json['sale_id'] = sale['sale_id']
        customer_json['customer_id'] = sale['deliver_to']['customer_id'] 
        customer_json['first_name'] = sale['deliver_to']['first_name'] 
        customer_json['last_name'] = sale['deliver_to']['last_name'] 
        customer_json['street'] = sale['deliver_to']['street'] 
        customer_json['city'] = sale['deliver_to']['city']   
        customer_json['state'] = sale['deliver_to']['state'] 
        customer_json['zip'] = sale['deliver_to']['zip']       
         
        customer_json_list.append(customer_json)
        line_item_id = 0
        
        for line_item in sale['line_items']:
               
                
            line_item_json = {}
            line_item_json['sale_id'] = sale['sale_id']
            line_item_id += 1
            line_item_json['line_item_id'] = line_item_id
            line_item_json['product_id'] = line_item['product_id']
            line_item_json['price'] = line_item['price'] 
            line_item_json['quantity'] = line_item['quantity'] 
            line_item_json['taxable'] = line_item['taxable'] 
                
            line_item_json_list.append(line_item_json)
            

            
    f = open("data/peak_sales.csv", "w")
    dw = csv.DictWriter(f, sale_json_list[0].keys())
    dw.writeheader()
    dw.writerows(sale_json_list)
    f.close()
    
    f = open("data/peak_customers.csv", "w")
    dw = csv.DictWriter(f, customer_json_list[0].keys())
    dw.writeheader()
    dw.writerows(customer_json_list)
    f.close()
    
    f = open("data/peak_stores.csv", "w")
    dw = csv.DictWriter(f, store_json_list[0].keys())
    dw.writeheader()
    dw.writerows(store_json_list)
    f.close()
      
    f = open("data/peak_line_items.csv", "w")
    dw = csv.DictWriter(f, line_item_json_list[0].keys())
    dw.writeheader()
    dw.writerows(line_item_json_list)
    f.close()
    


creator
    value: Peak Deliveries
timestamp
    value: 2020-10-04 02:00:00
file_name
    value: peak_sales_2020_10_03.json
version
    value: 17.11.7
legal
    value: Unauthorized use, duplication, or possession, blah, blah
sales
    [0]
        sale_id
            value: 5763728874
        sale_date
            value: 2020-10-03
        sub_total
            value: 12
        tax
            value: 0
        total_amount
            value: 12
        pickup_from
            location_id
                value: 12573
            name
                value: Acme Gourmet Meals
            street
                value: 3000 Telegraph Ave
            city
                value: Berkeley
            state
                value: CA
            zip
                value: 94705
        deliver_to
            customer_id
                value: 3728404
            first_name
                value: Darrelle
            last_name
                value: Dohrmann
            street
                va

            value: 48
        tax
            value: 0
        total_amount
            value: 48
        pickup_from
            location_id
                value: 12573
            name
                value: Acme Gourmet Meals
            street
                value: 3000 Telegraph Ave
            city
                value: Berkeley
            state
                value: CA
            zip
                value: 94705
        deliver_to
            customer_id
                value: 3729206
            first_name
                value: Ardyce
            last_name
                value: Lauderdale
            street
                value: 483 Pearson Point
            city
                value: Oakland
            state
                value: CA
            zip
                value: 94602
        line_items
            [0]
                product_id
                    value: 42314677
                price
                    value: 12
                quantity
                

            value: 72
        pickup_from
            location_id
                value: 12573
            name
                value: Acme Gourmet Meals
            street
                value: 3000 Telegraph Ave
            city
                value: Berkeley
            state
                value: CA
            zip
                value: 94705
        deliver_to
            customer_id
                value: 3729246
            first_name
                value: Candy
            last_name
                value: Ben
            street
                value: 1479 Bartillon Plaza
            city
                value: Oakland
            state
                value: CA
            zip
                value: 94602
        line_items
            [0]
                product_id
                    value: 42314677
                price
                    value: 12
                quantity
                    value: 3
                taxable
                    value: N
            [1]

            state
                value: CA
            zip
                value: 94607
        line_items
            [0]
                product_id
                    value: 42314677
                price
                    value: 12
                quantity
                    value: 3
                taxable
                    value: N
            [1]
                product_id
                    value: 42314679
                price
                    value: 12
                quantity
                    value: 1
                taxable
                    value: N
            [2]
                product_id
                    value: 42314782
                price
                    value: 12
                quantity
                    value: 3
                taxable
                    value: N
            [3]
                product_id
                    value: 42314783
                price
                    value: 12
                quantity
                    va

                value: 18373 Golf View Pass
            city
                value: Berkeley
            state
                value: CA
            zip
                value: 94702
        line_items
            [0]
                product_id
                    value: 42314677
                price
                    value: 12
                quantity
                    value: 2
                taxable
                    value: N
            [1]
                product_id
                    value: 42314780
                price
                    value: 12
                quantity
                    value: 1
                taxable
                    value: N
            [2]
                product_id
                    value: 42314782
                price
                    value: 12
                quantity
                    value: 2
                taxable
                    value: N
            [3]
                product_id
                    value: 42314783
      

                    value: 42314780
                price
                    value: 12
                quantity
                    value: 1
                taxable
                    value: N
            [3]
                product_id
                    value: 42314782
                price
                    value: 12
                quantity
                    value: 1
                taxable
                    value: N
    [75]
        sale_id
            value: 5763728894
        sale_date
            value: 2020-10-03
        sub_total
            value: 144
        tax
            value: 0
        total_amount
            value: 144
        pickup_from
            location_id
                value: 12573
            name
                value: Acme Gourmet Meals
            street
                value: 3000 Telegraph Ave
            city
                value: Berkeley
            state
                value: CA
            zip
                value: 94705
        deliver

                    value: 1
                taxable
                    value: N
            [2]
                product_id
                    value: 42314780
                price
                    value: 12
                quantity
                    value: 3
                taxable
                    value: N
            [3]
                product_id
                    value: 42314784
                price
                    value: 12
                quantity
                    value: 4
                taxable
                    value: N
    [90]
        sale_id
            value: 5763728666
        sale_date
            value: 2020-10-03
        sub_total
            value: 120
        tax
            value: 0
        total_amount
            value: 120
        pickup_from
            location_id
                value: 12573
            name
                value: Acme Gourmet Meals
            street
                value: 3000 Telegraph Ave
            city
           

In [34]:
my_extract_peak_sales_2020_nested_json_to_csv('data/peak_sales_2020_10_03.json')

#create a table for peak sales
connection.rollback()

query = """

drop table if exists peak_sales;
create table peak_sales (
  sale_id varchar(100),
  sale_date varchar(100),
  sub_total varchar(100),
  tax varchar(100),
  total_amount varchar(100)
);

copy peak_sales
from '/user/projects/project-3-peeti-ucberkeley/code/data/peak_sales.csv' delimiter ',' NULL '' csv header;


"""


cursor.execute(query)

connection.commit()

my_read_csv_file("data/peak_sales.csv", limit=10)


['sale_id', 'sale_date', 'sub_total', 'tax', 'total_amount']
['5763728874', '2020-10-03', '12', '0', '12']
['5763729036', '2020-10-03', '72', '0', '72']
['5763728904', '2020-10-03', '24', '0', '24']
['5763728973', '2020-10-03', '96', '0', '96']
['5763728757', '2020-10-03', '108', '0', '108']
['5763729051', '2020-10-03', '144', '0', '144']
['5763729153', '2020-10-03', '24', '0', '24']
['5763728608', '2020-10-03', '96', '0', '96']
['5763728696', '2020-10-03', '84', '0', '84']

Printed  10 lines of  98 total lines.


In [37]:
#create a table for peak customers
connection.rollback()

query = """

drop table if exists peak_customers;
create table peak_customers (
  
  sale_id varchar(100),
  customer_id varchar(100),
  first_name varchar(100),
  last_name varchar(100),
  street varchar(100),
  city varchar(100),
  state varchar(100),
  zip varchar(100)
);

copy peak_customers
from '/user/projects/project-3-peeti-ucberkeley/code/data/peak_customers.csv' delimiter ',' NULL '' csv header;


"""


cursor.execute(query)

connection.commit()

my_read_csv_file("data/peak_customers.csv", limit=10)


['sale_id', 'customer_id', 'first_name', 'last_name', 'street', 'city', 'state', 'zip']
['5763728874', '3728404', 'Darrelle', 'Dohrmann', '46 Farwell Terrace', 'Oakland', 'CA', '94609']
['5763729036', '3729309', 'Moria', 'Greenwood', '8803 Delaware Crossing', 'Berkeley', 'CA', '94705']
['5763728904', '3728508', 'Josiah', 'Hulett', '6755 Melby Plaza', 'Oakland', 'CA', '94612']
['5763728973', '3728534', 'Gayle', 'MacGarrity', '286 Onsgard Center', 'Berkeley', 'CA', '94703']
['5763728757', '3729188', 'Courtenay', 'Shirrell', '75 West Park', 'Emeryville', 'CA', '94608']
['5763729051', '3729276', 'Christian', 'Anyene', '869 Transport Crossing', 'Berkeley', 'CA', '94707']
['5763729153', '3729242', 'Linnell', 'Barr', '521 Fallview Alley', 'Oakland', 'CA', '94602']
['5763728608', '3728705', 'Benedick', 'Staneland', '3852 Laurel Park', 'Berkeley', 'CA', '94704']
['5763728696', '3729340', 'Lanni', 'Pickavant', '481 Moose Pass', 'Oakland', 'CA', '94609']

Printed  10 lines of  98 total lines.


In [38]:
#create a table for peak stores
connection.rollback()

query = """

drop table if exists peak_stores;
create table peak_stores (
  
  sale_id varchar(100),
  location_id varchar(100),
  name varchar(100),
  street varchar(100),
  city varchar(100),
  state varchar(100),
  zip varchar(100)
);

copy peak_stores
from '/user/projects/project-3-peeti-ucberkeley/code/data/peak_stores.csv' delimiter ',' NULL '' csv header;


"""


cursor.execute(query)

connection.commit()

my_read_csv_file("data/peak_stores.csv", limit=10)


['sale_id', 'location_id', 'name', 'street', 'city', 'state', 'zip']
['5763728874', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763729036', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763728904', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763728973', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763728757', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763729051', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763729153', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763728608', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763728696', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']

Printed  10 lines of  98 total lines.


In [39]:
#create a table for pline item
connection.rollback()

query = """

drop table if exists peak_line_items;
create table peak_line_items (
  
  sale_id varchar(100),
  line_item_id varchar(100),
  product_id varchar(100),
  price varchar(100),
  quantity varchar(100),
  taxable varchar(100)
);

copy peak_line_items
from '/user/projects/project-3-peeti-ucberkeley/code/data/peak_line_items.csv' delimiter ',' NULL '' csv header;


"""


cursor.execute(query)

connection.commit()

my_read_csv_file("data/peak_line_items.csv", limit=10)


['sale_id', 'line_item_id', 'product_id', 'price', 'quantity', 'taxable']
['5763728874', '1', '42314780', '12', '1', 'N']
['5763729036', '1', '42314677', '12', '1', 'N']
['5763729036', '2', '42314782', '12', '3', 'N']
['5763729036', '3', '42314784', '12', '2', 'N']
['5763728904', '1', '42314780', '12', '1', 'N']
['5763728904', '2', '42314784', '12', '1', 'N']
['5763728973', '1', '42314677', '12', '2', 'N']
['5763728973', '2', '42314780', '12', '2', 'N']
['5763728973', '3', '42314782', '12', '2', 'N']

Printed  10 lines of  353 total lines.


In [42]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select zip,
        sum(total_amount::numeric) as total_sales, 
       sum(total_amount::numeric)*0.18 as cut_paid_to_peak,
       sum(total_amount::numeric)*(1-0.18) as net_to_agm
from peak_sales a
join peak_stores b
on a.sale_id = b.sale_id
group by 1
order by 4 desc ;

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,zip,total_sales,cut_paid_to_peak,net_to_agm
0,94705,6480,1166.4,5313.6


# search with station has highest population

In [44]:
from geographiclib.geodesic import Geodesic

In [70]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

def my_station_get_zips(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    print("\n-------------------------------------------------------------------------------")
    print("  Zip Codes within " + str(miles) + " mile(s) of " + station + " BART Station")
    print("-------------------------------------------------------------------------------\n")
    
    total_population = 0
    
    for row in rows:
        zip = row[0]
        population = row[1]
        print("     zip:", zip, "  population: ", f'{population:10,}')
        total_population += population
        
    
    print("\n-------------------------------------------------------------------------------")
    print("  Total Population: ", f'{total_population:10,}')
    print("-------------------------------------------------------------------------------")
    
def my_neo4j_shortest_path(from_station, to_station):
    "given a from station and to station, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)

    query = """

    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_station, target=to_station)
    
    for r in result:
        
        total_cost = int(r['totalCost'])
        
        print("\n--------------------------------")
        print("   Total Cost: ", total_cost)
        print("   Minutes: ", round(total_cost / 60.0,1))
        print("--------------------------------")
        
        nodes = r['nodes']
        costs = r['costs']
        
        i = 0
        previous = 0
        
        for n in nodes:
            
            print(n + ", " + str(int(costs[i]) - previous)  + ", " + str(int(costs[i])))
            
            previous = int(costs[i])
            i += 1
    

# within 5 miles radius

In [67]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select station from stations group by 1;

"""

station_name = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
l = [list(i) for i in np.array(list(station_name.values))]

station_name_list = [item for sublist in l for item in sublist]
station_name_list

['Bay Fair',
 'Fremont',
 'SFO',
 'Pittsburg',
 'Richmond',
 'Fruitvale',
 'Hayward',
 'Ashby',
 'Downtown Berkeley',
 'Lafayette',
 'Dublin',
 'MacArthur',
 'El Cerrito Plaza',
 'Walnut Creek',
 'San Bruno',
 'Rockridge',
 'North Berkeley',
 'Daly City',
 'North Concord',
 'Milpitas',
 'Union City',
 'Colma',
 '24th Street Mission',
 'South San Francisco',
 'Pittsburg Center',
 'El Cerrito del Norte',
 'OAK',
 'Powell Street',
 'Lake Merritt',
 'Concord',
 '16th Street Mission',
 'Coliseum',
 'Antioch',
 '12th Street',
 'Civic Center',
 'Warm Springs',
 'Millbrae',
 'San Leandro',
 'Berryessa',
 'Pleasant Hill',
 'Embarcadero',
 'Castro Valley',
 'West Oakland',
 'Balboa Park',
 'West Dublin',
 'Orinda',
 '19th Street',
 'Glen Park',
 'South Hayward',
 'Montgomery Street']

In [69]:
[my_station_get_zips(k, 5) for k in station_name_list] 


-------------------------------------------------------------------------------
  Zip Codes within 5 mile(s) of Bay Fair BART Station
-------------------------------------------------------------------------------

     zip: 94541   population:      66,056
     zip: 94542   population:      14,217
     zip: 94544   population:      79,235
     zip: 94546   population:      44,272
     zip: 94577   population:      48,088
     zip: 94578   population:      41,059
     zip: 94579   population:      22,040
     zip: 94580   population:      29,942
     zip: 94603   population:      34,593
     zip: 94605   population:      43,112
     zip: 94621   population:      35,287

-------------------------------------------------------------------------------
  Total Population:     457,901
-------------------------------------------------------------------------------

-------------------------------------------------------------------------------
  Zip Codes within 5 mile(s) of Fremont BART Sta


-------------------------------------------------------------------------------
  Zip Codes within 5 mile(s) of San Bruno BART Station
-------------------------------------------------------------------------------

     zip: 94005   population:       4,692
     zip: 94010   population:      42,730
     zip: 94014   population:      49,515
     zip: 94015   population:      64,887
     zip: 94030   population:      22,710
     zip: 94044   population:      39,065
     zip: 94066   population:      43,124
     zip: 94080   population:      67,597
     zip: 94128   population:         135

-------------------------------------------------------------------------------
  Total Population:     334,455
-------------------------------------------------------------------------------

-------------------------------------------------------------------------------
  Zip Codes within 5 mile(s) of Rockridge BART Station
----------------------------------------------------------------------------

     zip: 94108   population:      14,550
     zip: 94109   population:      56,677
     zip: 94110   population:      74,161
     zip: 94111   population:       3,620
     zip: 94112   population:      85,166
     zip: 94114   population:      34,754
     zip: 94115   population:      35,004
     zip: 94116   population:      47,823
     zip: 94117   population:      44,239
     zip: 94118   population:      41,468
     zip: 94121   population:      43,795
     zip: 94122   population:      62,131
     zip: 94123   population:      25,941
     zip: 94124   population:      35,363
     zip: 94127   population:      21,423
     zip: 94129   population:       3,994
     zip: 94130   population:       3,064
     zip: 94131   population:      29,902
     zip: 94132   population:      31,488
     zip: 94133   population:      26,527
     zip: 94134   population:      42,471
     zip: 94158   population:       7,291

---------------------------------------------------------------------------

     zip: 94107   population:      29,689
     zip: 94108   population:      14,550
     zip: 94109   population:      56,677
     zip: 94110   population:      74,161
     zip: 94111   population:       3,620
     zip: 94114   population:      34,754
     zip: 94115   population:      35,004
     zip: 94116   population:      47,823
     zip: 94117   population:      44,239
     zip: 94118   population:      41,468
     zip: 94122   population:      62,131
     zip: 94123   population:      25,941
     zip: 94124   population:      35,363
     zip: 94127   population:      21,423
     zip: 94129   population:       3,994
     zip: 94130   population:       3,064
     zip: 94131   population:      29,902
     zip: 94132   population:      31,488
     zip: 94133   population:      26,527
     zip: 94158   population:       7,291

-------------------------------------------------------------------------------
  Total Population:     698,612
-----------------------------------------------

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

# BART Stations that have highest population 
* Montgomery Street = 826,249 ** 
* Glen Park = 986,074 ** 
* 24th Street Mission = 989,138 **
* Balboa Park =  936,912
* Embarcadero = 698,612
* Civic Center = 870,044
* 16th Street Mission = 870,044
* Powell Street = 870,044
* Daly City = 805,122

In [73]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","w205"))
session = driver.session(database="neo4j")
my_neo4j_shortest_path('depart Downtown Berkeley' , 'arrive Montgomery Street')


--------------------------------
   Total Cost:  1500
   Minutes:  25.0
--------------------------------
depart Downtown Berkeley, 0, 0
red Downtown Berkeley, 0, 0
red Ashby, 180, 180
red MacArthur, 240, 420
red 19th Street, 180, 600
red 12th Street, 120, 720
red West Oakland, 300, 1020
red Embarcadero, 420, 1440
red Montgomery Street, 60, 1500
arrive Montgomery Street, 0, 1500


In [74]:
my_neo4j_shortest_path('depart Downtown Berkeley' , 'arrive Glen Park')


--------------------------------
   Total Cost:  2160
   Minutes:  36.0
--------------------------------
depart Downtown Berkeley, 0, 0
red Downtown Berkeley, 0, 0
red Ashby, 180, 180
red MacArthur, 240, 420
red 19th Street, 180, 600
red 12th Street, 120, 720
red West Oakland, 300, 1020
red Embarcadero, 420, 1440
red Montgomery Street, 60, 1500
red Powell Street, 120, 1620
red Civic Center, 60, 1680
red 16th Street Mission, 180, 1860
red 24th Street Mission, 120, 1980
red Glen Park, 180, 2160
arrive Glen Park, 0, 2160


In [75]:
my_neo4j_shortest_path('depart Downtown Berkeley' , 'arrive 24th Street Mission')



--------------------------------
   Total Cost:  1980
   Minutes:  33.0
--------------------------------
depart Downtown Berkeley, 0, 0
red Downtown Berkeley, 0, 0
red Ashby, 180, 180
red MacArthur, 240, 420
red 19th Street, 180, 600
red 12th Street, 120, 720
red West Oakland, 300, 1020
red Embarcadero, 420, 1440
red Montgomery Street, 60, 1500
red Powell Street, 120, 1620
red Civic Center, 60, 1680
red 16th Street Mission, 180, 1860
red 24th Street Mission, 120, 1980
arrive 24th Street Mission, 0, 1980


In [76]:
my_neo4j_shortest_path('depart Downtown Berkeley' , 'arrive Balboa Park')



--------------------------------
   Total Cost:  2280
   Minutes:  38.0
--------------------------------
depart Downtown Berkeley, 0, 0
red Downtown Berkeley, 0, 0
red Ashby, 180, 180
red MacArthur, 240, 420
red 19th Street, 180, 600
red 12th Street, 120, 720
red West Oakland, 300, 1020
red Embarcadero, 420, 1440
red Montgomery Street, 60, 1500
red Powell Street, 120, 1620
red Civic Center, 60, 1680
red 16th Street Mission, 180, 1860
red 24th Street Mission, 120, 1980
red Glen Park, 180, 2160
red Balboa Park, 120, 2280
arrive Balboa Park, 0, 2280


In [77]:
my_neo4j_shortest_path('depart Downtown Berkeley' , 'arrive Embarcadero')



--------------------------------
   Total Cost:  1440
   Minutes:  24.0
--------------------------------
depart Downtown Berkeley, 0, 0
red Downtown Berkeley, 0, 0
red Ashby, 180, 180
red MacArthur, 240, 420
red 19th Street, 180, 600
red 12th Street, 120, 720
red West Oakland, 300, 1020
red Embarcadero, 420, 1440
arrive Embarcadero, 0, 1440


In [78]:
my_neo4j_shortest_path('depart Downtown Berkeley' , 'arrive Civic Center')



--------------------------------
   Total Cost:  1680
   Minutes:  28.0
--------------------------------
depart Downtown Berkeley, 0, 0
red Downtown Berkeley, 0, 0
red Ashby, 180, 180
red MacArthur, 240, 420
red 19th Street, 180, 600
red 12th Street, 120, 720
red West Oakland, 300, 1020
red Embarcadero, 420, 1440
red Montgomery Street, 60, 1500
red Powell Street, 120, 1620
red Civic Center, 60, 1680
arrive Civic Center, 0, 1680


In [79]:
my_neo4j_shortest_path('depart Downtown Berkeley' , 'arrive 16th Street Mission')



--------------------------------
   Total Cost:  1860
   Minutes:  31.0
--------------------------------
depart Downtown Berkeley, 0, 0
red Downtown Berkeley, 0, 0
red Ashby, 180, 180
red MacArthur, 240, 420
red 19th Street, 180, 600
red 12th Street, 120, 720
red West Oakland, 300, 1020
red Embarcadero, 420, 1440
red Montgomery Street, 60, 1500
red Powell Street, 120, 1620
red Civic Center, 60, 1680
red 16th Street Mission, 180, 1860
arrive 16th Street Mission, 0, 1860


In [80]:
my_neo4j_shortest_path('depart Downtown Berkeley' , 'arrive Powell Street')



--------------------------------
   Total Cost:  1620
   Minutes:  27.0
--------------------------------
depart Downtown Berkeley, 0, 0
red Downtown Berkeley, 0, 0
red Ashby, 180, 180
red MacArthur, 240, 420
red 19th Street, 180, 600
red 12th Street, 120, 720
red West Oakland, 300, 1020
red Embarcadero, 420, 1440
red Montgomery Street, 60, 1500
red Powell Street, 120, 1620
arrive Powell Street, 0, 1620


In [83]:
my_neo4j_shortest_path('depart Downtown Berkeley' , 'arrive Daly City')


--------------------------------
   Total Cost:  2520
   Minutes:  42.0
--------------------------------
depart Downtown Berkeley, 0, 0
red Downtown Berkeley, 0, 0
red Ashby, 180, 180
red MacArthur, 240, 420
red 19th Street, 180, 600
red 12th Street, 120, 720
red West Oakland, 300, 1020
red Embarcadero, 420, 1440
red Montgomery Street, 60, 1500
red Powell Street, 120, 1620
red Civic Center, 60, 1680
red 16th Street Mission, 180, 1860
red 24th Street Mission, 120, 1980
red Glen Park, 180, 2160
red Balboa Park, 120, 2280
red Daly City, 240, 2520
arrive Daly City, 0, 2520
